# General

- Include the notebook AND a pdf of the notebook as part of your homework. 
- label this as HW2\Ryslik_1_Coding.pdf

# Part 1 - PCA

- Load the IRIS data set on flowers (we do that for you).
- Write a function that takes in a matrix and provides you the eigenvalues and eigenvectors (using numpy).
- Project your centered data into two dimensional space and plot the projected points. Color code the points by flower type. 
- Show that you get the same eigenvalues/eigenvectors/projection if you use scikit's PCA or you manually do the projection via numpy (you can use numpy to compute the eigenvectors and eigenvalues)
- Can you make a good rule of what type of flower it is by just using the first two principle components? Describe how?
- What percentage of the total data variance do the first two principle components capture.

In [1]:
# Data prep
from sklearn.datasets import load_iris
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
from scipy.linalg import fractional_matrix_power
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
import warnings
warnings.simplefilter("ignore", np.ComplexWarning)

iris = load_iris() #see the documentation here: https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html
#plt.scatter(x = dataset[0,:], y = dataset[1,:], c=iris.target)

In [2]:
# Display iris data such that my brain actually likes it.
iris_b = load_iris(as_frame=True)
iris_b.data

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                  5.1               3.5                1.4               0.2
1                  4.9               3.0                1.4               0.2
2                  4.7               3.2                1.3               0.2
3                  4.6               3.1                1.5               0.2
4                  5.0               3.6                1.4               0.2
..                 ...               ...                ...               ...
145                6.7               3.0                5.2               2.3
146                6.3               2.5                5.0               1.9
147                6.5               3.0                5.2               2.0
148                6.2               3.4                5.4               2.3
149                5.9               3.0                5.1               1.8

[150 rows x 4 columns]

In [3]:
# Write a function that takes in a matrix and provides you the eigenvalues and eigenvectors, using numpy.
def get_eigen_data(data):
    # Get Correlation Coeff.
    coeff = np.corrcoef(data)
    
    # Get Scaler & Mean.
    scaler = preprocessing.StandardScaler().fit(data.T)
    
    # Get Centered Dataset.
    iris_centered = data - scaler.mean_[:,None]
    print("Centered Iris Dims: " + str(iris_centered.shape))
    #plt.scatter(x = iris_centered[0,:], y = iris_centered[1,:], c=iris.target)
    
    # Take Dot Product
    np.dot(iris_centered, iris_centered.T)/iris_centered.shape[1]
    
    # Grab Eigenvectors & Eigenvalues.
    eigen_values, eigen_vectors = np.linalg.eig(np.cov(iris_centered, rowvar=True, bias=True).T)
    
    # Sort the eigen data.
    sort = eigen_values.argsort()[::-1]    
    eigen_values = eigen_values[sort]
    eigen_vectors = eigen_vectors[:,sort]
    
    # Return the data.
    return eigen_values, eigen_vectors, iris_centered

# Call Function.  
dataset = iris.data.copy().T
eig_vals, eig_vects, iris_centered = get_eigen_data(dataset)

# Print Eigen Data
print("Eigenvector: \n",eig_vects,"\n")
print("Eigenvalues: \n", eig_vals, "\n")

Centered Iris Dims: (4, 150)
Eigenvector: 
 [[ 0.36138659 -0.65658877 -0.58202985  0.31548719]
 [-0.08452251 -0.73016143  0.59791083 -0.3197231 ]
 [ 0.85667061  0.17337266  0.07623608 -0.47983899]
 [ 0.3582892   0.07548102  0.54583143  0.75365743]] 

Eigenvalues: 
 [4.20005343 0.24105294 0.0776881  0.02367619] 



In [4]:
# Project your centered data into two dimensional space and plot the projected points. Color code the points by flower type.
pca_manual = np.dot(eig_vects.T, iris_centered)
plt.scatter(x = pca_manual[0,:], y = pca_manual[1,:], c=iris.target)

In [5]:
# Show that you get the same eigenvalues/eigenvectors/projection if you use scikit's PCA or you manually do the projection via numpy (you can use numpy to compute the eigenvectors and eigenvalues)
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

# Start doin stuff.
pca.fit_transform(dataset.T)
#auto_data.shape

auto_pca = pca.fit_transform(dataset.T)
#auto_pca.shape
plt.scatter(x = auto_pca[:,0], y = auto_pca[:,1], c=iris.target)

### Can you make a good rule of what type of flower it is by just using the first two principle components? Describe how?
- Yes you can, most of the varience is in the first two components anyways ([0.92461872 0.05306648 0.01710261 0.00521218]), so the presence of the prior two components doesn't change anything significantly.


In [6]:
# What percentage of the total data variance do the first two principle components capture.
l_ratio = pca.explained_variance_ratio_[0] + pca.explained_variance_ratio_[1]
print("{0:.4%}".format(l_ratio))

97.7685%


# Part 2 - Linear Regression

- Load the Diabetes dataset (we do that for you)
- Break the data set up into the first 350 observations for training and the remaining observations for test.
- Write a function that takes in a feature matrix and a y-vector and fits the line of best fit using least squares regression. 
- Fit the line of best fit on the training data. 
- Make a prediction on the test data. 
- Compute the total Sum of Squared Error (SSE) and then compute the MSE on the test data.
- Fit the linear regression model using the stats models package as in class.
- Which variables are statistically significant at the 5% level? 
- What is the value of R^2? How is that interpreted?

In [7]:
from sklearn.datasets import load_diabetes
import pandas as pd
import statsmodels.api as sm
import io
diabetes = load_diabetes(as_frame=True)

In [8]:
diabetes['target']

0      151.0
1       75.0
2      141.0
3      206.0
4      135.0
       ...  
437    178.0
438    104.0
439    132.0
440    220.0
441     57.0
Name: target, Length: 442, dtype: float64

In [9]:
diabetes.feature_names

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

In [10]:
# Break the data set up into the first 350 observations for training and the remaining observations for test.
training_data = diabetes["data"][0:350]
testing_data = diabetes["data"][350:442]

In [11]:
# Write a function that takes in a feature matrix and a y-vector and fits the line of best fit using least squares regression.
def get_best_line(feat_mat, y_vect):

    # Add the constant.
    feat_mat = sm.add_constant(feat_mat)

    # Get and fit the model.
    model = sm.OLS(y_vect, feat_mat)
    results = model.fit()
    print(results.summary())
    #sm.graphics.plot_fit(results,1, vlines=False);

    # Get the coeffs and the std err values.
    coeffs = results.params
    std_errs = results.bse

    # Return the line of best fit, fit to the data.
    # Adding the constant.
    y_hat = coeffs[0]

    for i in range(1, len(coeffs)):
        y_hat += coeffs[i] * std_errs[i]
    return y_hat

# Set the variables & call the function.
y_vect = diabetes.target[0:350]
y_hat = get_best_line(training_data, y_vect)


                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.504
Model:                            OLS   Adj. R-squared:                  0.490
Method:                 Least Squares   F-statistic:                     34.48
Date:                Sun, 06 Mar 2022   Prob (F-statistic):           5.38e-46
Time:                        23:29:19   Log-Likelihood:                -1890.2
No. Observations:                 350   AIC:                             3802.
Df Residuals:                     339   BIC:                             3845.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        152.0799      2.917     52.137      0.0

In [12]:
# Fit the line of best fit on the training data.
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
results = reg.fit(training_data, y_vect)
print(results.coef_)

[ -10.82642893 -249.48428541  514.40863256  295.22963645 -456.83095843
  201.72327107  -48.34518664  138.29973577  631.97471559   87.89687854]


In [13]:
# Make a prediction on the test data.
predict = reg.predict(testing_data)
predict

array([249.73713513,  87.3519183 ,  59.67330031, 171.13303685,
       194.2613731 , 132.80131482,  91.65414427, 199.47376009,
        56.22447271, 172.59442294, 195.79655277, 120.7542843 ,
       231.61334653, 163.35726562, 160.64716435, 164.37765919,
       254.51873453, 254.62729539, 200.47316451, 185.14295451,
        60.08369427, 200.85524825, 108.87625578, 143.36981529,
       128.32486751, 175.35457459, 210.55490963, 165.22427363,
       161.79816651, 137.0856577 , 173.82224504,  74.91906646,
       253.95371552, 113.90067393, 109.77297524, 137.33085478,
       110.49759527,  97.25697617, 159.26143257,  75.30981169,
       258.58231308,  58.36486635, 100.02606938, 102.77880435,
       268.48641391, 168.97487081,  65.38198998, 183.11087173,
       171.23843967, 187.11698446, 183.26703537,  90.37211222,
       147.07960251, 252.78454669, 197.65498095, 278.00768364,
        52.21933948, 174.95631999, 202.35130825, 170.31365556,
       157.37468888, 152.92011903, 231.18250209, 123.37

In [14]:
# Compute the total Sum of Squared Error (SSE) and then compute the MSE on the test data.
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_vect[0:92], predict)
#y_vect.shape
mse

6772.493862234267

### Which variables are statistically significant at the 5% level?
- Sex
- BMI
- Blood Pressure
- S5
### What's the value of R-squared? How is this interpreted?
R-squared = 0.504. This is basically a measure of how well the model fits the data. In a vacuum, the closer to one, the better.